In [1]:
from flask import Flask, render_template, request,jsonify
# from flask_cors import CORS,cross_origin
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as uReq
import pandas as pd


custname=[]
prorating=[]
revhead=[]
revdeatil=[]

flipkart_url = "https://www.flipkart.com/search?q=samsung+a8"

uClient = uReq(flipkart_url) # requesting the webpage from the internet
flipkartPage = uClient.read() # reading the webpage
uClient.close() # closing the connection to the web server
flipkart_html = bs(flipkartPage, "html.parser") # parsing the webpage as HTML


bigboxes = flipkart_html.findAll("div", {"class": "_1UoZlX"})

box=bigboxes[0]

product_link=box.a['href']

productLink=flipkart_url+product_link
prodRes = requests.get(productLink)

prod_html = bs(prodRes.text, "html.parser")

baseurl='https://www.flipkart.com'
all_rev_link = prod_html.find('div', {'class': "swINJg _3nrCtb"})

all_rev_link=all_rev_link.find_parent().get('href')


i=1
url=baseurl+all_rev_link+'&page='+str(i)

while True:
    
    url=baseurl+all_rev_link+'&page='+str(i)
    page=requests.get(url)
    if page.status_code != 200:
        break
        
    
    all_review_page = requests.get(url)

    all_review_html = bs(all_review_page.text, "html.parser")
    
    review_container=all_review_html.findAll('div',{'class':'col _390CkK _1gY8H-'})
    
    for review in review_container:
        name=review.findAll('p',{'class':'_3LYOAd _3sxSiS'})[0].text
        custname.append(name)
    
        rating=review.findAll('div',{'class':['hGSR34 E_uFuv','hGSR34 _1x2VEC E_uFuv','hGSR34 _1nLEql E_uFuv']})[0].text    
        prorating.append(rating)
    
        review_header=review.findAll('p',{'class':'_2xg6Ul'})[0].text
        revhead.append(review_header)
    
        review_detail=review.findAll('div',{'class':''})[0].text
        revdeatil.append(review_detail)
        
    i = i+1
    
reviews=pd.DataFrame(
                        {
                            'Customer Name':custname,
                            'Rating':prorating,
                            'REview Head':revhead,
                            "Review Detail" : revdeatil
                        })


reviews

,Customer Name,Rating,REview Head,Review Detail
0,Soumya Guha Roy,4,Beauty & the beast combined.,I have been using the device since last 3 days...
1,deepak pagar nashik,5,"superb ,wesome,stylish, powerful ,metal finish...",i bought this stunning metal body slim phone b...
2,Rejo Raju,5,AWESOME !,I have Bought this phone on September and its ...
3,Archita Jalan,5,Awesome Phone !!,1]Slimmest Phone in this range.2]Value for Mon...
4,Arun Nath S,5,The thinnest Samsung,"Well, nothing more to add than what other have..."
...,...,...,...,...
237,mayank agrawal,1,dont trust these people,this is a fraud sellerthey have sent me a comp...
238,Sagar Ganmote,1,Not recommended at all,3rd class phoneREAD MORE
239,Tanya Choudhury,5,Brilliant,Is this have a exchange option with J7READ MORE
240,Sayanka Saha,5,sim slot,is the phone optional sim slot?i want to buy t...
